In [ ]:
import os
from keras.applications.imagenet_utils import preprocess_input
 
from matplotlib.pyplot import imshow
 
import numpy as np
from keras import layers
from keras.layers import *
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.optimizers import *
from keras.utils.vis_utils import model_to_dot
# from keras.utils import plot_model
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import keras.backend as K
from keras import Sequential
from keras.layers import Softmax,Activation,Dense
from keras.utils.np_utils import to_categorical

from sklearn.metrics import recall_score,f1_score,precision_score
from keras.applications.imagenet_utils import preprocess_input
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
 
 
 
 

In [ ]:
l_0=list(os.listdir('dataset0/0'))
l_1=list(os.listdir('dataset0/1'))
l_2=list(os.listdir('dataset0/2'))
l_3=list(os.listdir('dataset0/3'))
l_4=list(os.listdir('dataset0/4'))

In [ ]:
print(len(l_0),len(l_1),len(l_2),len(l_3),len(l_4))

In [ ]:
image_types = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")
def list_images(basePath, contains=None):
    # 返回有效的图片路径数据集
    return list_files(basePath, validExts=image_types, contains=contains)
def list_files(basePath, validExts=None, contains=None):
    # 遍历图片数据目录，生成每张图片的路径
    for (rootDir, dirNames, filenames) in os.walk(basePath):
        # 循环遍历当前目录中的文件名
        for filename in filenames:
            if contains is not None and filename.find(contains) == -1:
                continue
            ext = filename[filename.rfind("."):].lower()
            if validExts is None or ext.endswith(validExts):
                imagePath = os.path.join(rootDir, filename)
                yield imagePath

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
import cv2
import os
from keras.utils.np_utils import to_categorical

In [ ]:
print("------开始读取数据------")
data = []
labels = []

imagePaths = sorted(list(list_images('dataset0')))
random.seed(0)
random.shuffle(imagePaths)
for imagePath in imagePaths:
    image = cv2.imread(imagePath)
    data.append(image)
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
    if len(data)==1000:
        break
print(len(data))
print("------读取文件结束------")

In [ ]:
len(data)

In [ ]:
plt.imshow(data[100])

In [ ]:
labels[10]

In [ ]:
labels

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.1, random_state=88)

In [ ]:
plt.imshow(trainX[50])

In [ ]:
trainY[50]

In [ ]:

trainX=np.array(trainX)
testX=np.array(testX)
trainX = trainX.astype('float32')
testX = testX.astype('float32')
trainX /= 255
testX /= 255
trainY=np.array(trainY)

testY=np.array(testY)
print(type(trainX))
print(type(trainY))
print(type(testX))
print(type(testY))

In [ ]:
trainY=to_categorical(trainY,num_classes=5)
testY=to_categorical(testY,num_classes=5)
print(trainX.shape) 
print(trainY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:
trainX

In [ ]:
def VGG_13(X,Y):
 
    model = Sequential()
    model.add(Conv2D(64,(3,3),strides=(1,1),input_shape=(64,64,3),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(128,(3,2),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(BatchNormalization())

    model.add(Conv2D(128,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(Conv2D(512,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(4096,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(5,activation='softmax'))
    model.summary()
    return model

In [ ]:
import keras.backend as K
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
model=VGG_13(trainX,trainY)
 
model.compile("adam","categorical_crossentropy", metrics=['accuracy'])

def scheduler(epoch):
    # 每隔10个epoch，学习率减小为原来的1/10
    if epoch % 10 == 0 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.1)
        print("lr changed to {}".format(lr * 0.1))
    return K.get_value(model.optimizer.lr)

reduce_lr = LearningRateScheduler(scheduler)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)


In [ ]:
 hist=model.fit(trainX, trainY, batch_size=32, epochs=50, 
                callbacks=[reduce_lr],
    #                reduce_lr,,early_stopping
                validation_data=(testX,testY))

In [ ]:
costrain,acctr=model.evaluate(trainX, trainY, batch_size=64)
cost,acc = model.evaluate(testX, testY, batch_size=64)
print ("训练集准确 = " , acctr)
print ("测试集准确度 = " ,acc)



In [ ]:
plt.figure(figsize=(20,8))

plt.subplot(1, 2, 1)
plt.plot(hist.history.get("accuracy")[0:],"green")
plt.plot(hist.history.get("val_accuracy")[0:],color='#054E9F')
plt.ylabel("acc")
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.grid(alpha=1,linestyle=':')

plt.subplot(1, 2, 2)
plt.plot(hist.history.get("loss")[0:],"green")
plt.plot(hist.history.get("val_loss")[0:],color='#054E9F')
plt.ylabel("loss")
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.grid(alpha=1,linestyle=':')

In [ ]:
predY = model.predict(testX)
fpr, tpr, thresholds_keras = roc_curve(testY[:,0], predY[:,0])
auc_val = auc(fpr, tpr)
print("AUC : ",auc_val)
plt.figure()
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='AUC = {:.3f})'.format(auc_val))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()